## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/dl_pipeline')
from dl_pipeline import train_dl_complete

## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_dl_'
model = 'breslow'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'breslow'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
model: breslow
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6029402680832664
Integrated Brier Score: 0.17530893421383814
{'model': 'breslow', 'dataset': 'METABRIC', 'cindex_train': [0.6264538878887456], 'cindex_test': [0.6029402680832664], 'ibs_train': [0.16273263044626768], 'ibs_test': [0.17530893421383814]}
1
model: breslow
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5712447990995155
Integrated Brier Score: 0.1828003838047725
{'model': 'breslow', 'dataset': 'METABRIC', 'cindex_train': [0.5966161112542747], 'cindex_test': [0.5712447990995155], 'ibs_train': [0.1694391372839318], 'ibs_test': [0.1828003838047725]}
2
model: breslow
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5694293020397645
Integrated Brier Score: 0

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,breslow,METABRIC,0.626454,0.602940,0.162733,0.175309
1,breslow,METABRIC,0.596616,0.571245,0.169439,0.182800
2,breslow,METABRIC,0.609052,0.569429,0.170942,0.185386
3,breslow,METABRIC,0.644701,0.627387,0.161571,0.174761
4,breslow,METABRIC,0.634120,0.624115,0.162662,0.174837
5,breslow,FLCHAIN,0.795047,0.802032,0.101139,0.102433
6,breslow,FLCHAIN,0.801971,0.776686,0.099843,0.106505
7,breslow,FLCHAIN,0.787867,0.793478,0.102210,0.101114
8,breslow,FLCHAIN,0.793960,0.805745,0.099804,0.093554
9,breslow,FLCHAIN,0.799086,0.782149,0.099042,0.100716


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'breslow'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
model: breslow
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5204419889502763
Integrated Brier Score: 0.25618603458032035
{'model': 'breslow', 'dataset': 'BLCA', 'cindex_train': [0.5968877908477451], 'cindex_test': [0.5204419889502763], 'ibs_train': [0.21920704673786306], 'ibs_test': [0.25618603458032035]}
1
model: breslow
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.4860731840524304
Integrated Brier Score: 0.21796787946578938
{'model': 'breslow', 'dataset': 'BLCA', 'cindex_train': [0.5315154887775922], 'cindex_test': [0.4860731840524304], 'ibs_train': [0.21423945522172516], 'ibs_test': [0.21796787946578938]}
2
model: breslow
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.48446833930704897
Integrated Brier Score: 0.2976369

In [11]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'tcga_results.csv',index=False)

In [9]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,breslow,BLCA,0.596888,0.520442,0.219207,0.256186
1,breslow,BLCA,0.531515,0.486073,0.214239,0.217968
2,breslow,BLCA,0.523892,0.484468,0.240004,0.297637
3,breslow,BLCA,0.511759,0.533792,0.221049,0.219300
4,breslow,BLCA,0.708730,0.509373,0.165424,0.270782
5,breslow,BRCA,0.508099,0.444444,0.181600,0.204575
6,breslow,BRCA,0.505325,0.461476,0.187872,0.197669
7,breslow,BRCA,0.519031,0.518657,0.187462,0.210744
8,breslow,BRCA,0.508708,0.461136,0.194102,0.178941
9,breslow,BRCA,0.575571,0.518953,0.184305,0.195450
